# Dependencies installation

In [1]:
!pip install --upgrade numpy==1.16.4
!pip install --upgrade pandas==1.0.5
!pip install --upgrade "ibm-watson>=5.3.0"

  Attempting uninstall: PyJWT
    Found existing installation: PyJWT 1.7.1
    Uninstalling PyJWT-1.7.1:
      Successfully uninstalled PyJWT-1.7.1


# Utilities

In [151]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# Your input here

In [66]:
_WATSON_NLU_APIKEY = 'HzTEJ03k2tryP4RcSauPmFL_8Xzdqyya7Lr0qSGeQYbd'
_WATSON_NLU_URL = 'https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com'    #more details here - https://cloud.ibm.com/apidocs/natural-language-understanding?code=python#endpoint-cloud
                  
keyword = ['delta']
site = 'https://www.who.int/news/item/28-11-2021-update-on-omicron'    #website to analyze - english website

# Watson NLU interaction here

In [155]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions, MetadataOptions

                                          
authenticator = IAMAuthenticator(_WATSON_NLU_APIKEY)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2021-08-01',
    authenticator=authenticator
)

natural_language_understanding.set_service_url(_WATSON_NLU_URL)

from ibm_watson import ApiException
try:
    response = natural_language_understanding.analyze(
        url=site,
    features=Features(metadata=MetadataOptions(), sentiment=SentimentOptions(document=True, targets=keyword), emotion=EmotionOptions(document=True))).get_result()

    out_string = json.dumps(response, indent=2)
    # build CSV
    # site, keyword, etc.

    import pandas as pd
    out_json = json.loads(out_string)    #dict
    #print(out_string)
except ApiException as ex:
    print("Method failed with status code " + str(ex.code) + ": " + ex.message)




# Generate csv and data download link

In [163]:
data = {'site':site, 'title': out_json['metadata']['title'], 'publication_date': out_json['metadata']['publication_date'], 'keyword': keyword,
        'sentiment.target.text': out_json['sentiment']['targets'][0]['text'], 'sentiment.target.score': out_json['sentiment']['targets'][0]['score'], 'sentiment.target.label': out_json['sentiment']['targets'][0]['label'],
        'sentiment.document.score': out_json['sentiment']['document']['score'], 'sentiment.document.label': out_json['sentiment']['document']['label'],
        'emotion.document.emotion.sadness': out_json['emotion']['document']['emotion']['sadness'],'emotion.document.emotion.joy': out_json['emotion']['document']['emotion']['joy'],
        'emotion.document.emotion.fear': out_json['emotion']['document']['emotion']['fear'],'emotion.document.emotion.disgust': out_json['emotion']['document']['emotion']['disgust'],
        'emotion.document.emotion.anger': out_json['emotion']['document']['emotion']['anger']
       }
  
# Creates pandas DataFrame.  
df = pd.DataFrame(data)
df.set_index('site', inplace=True)
  
# print the data  
df

# generate download link
create_download_link(df, filename=site + '_' + keyword[0] + '.csv')